Generador de vectores demográficos

In [3]:
import csv

# Función para cargar tipos de usuario desde el archivo
def load_user_types(filepath):
    types = []
    with open(filepath, 'r', newline='') as file:
        reader = csv.reader(file, delimiter='\t')
        for row in reader:
            types.append({
                'tipo': row[0],
                'edad_min': int(row[1]),
                'edad_max': int(row[2]),
                'sexo': row[3].lower(),  # Asegurarse de que todo está en minúsculas para la comparación
                'profesion': row[4].lower()  # Asegurarse de que todo está en minúsculas para la comparación
            })
    return types

# Profesiones agrupadas
professions = {
    'c': ['artist', 'writer', 'entertainment', 'educator', 'librarian'],
    'b': ['executive', 'administrator', 'salesman', 'marketing', 'lawyer'],
    'm': ['doctor', 'healthcare', 'scientist'],
    't': ['technician', 'programmer', 'engineer'],
    'o': ['student', 'none', 'other', 'homemaker', 'retired']
}

# Función para determinar el tipo de usuario
def determine_user_type(age, sex, profession, types):
    for type_info in types:
        # Verifica edad
        if type_info['edad_min'] <= age <= type_info['edad_max']:
            # Verifica sexo
            
            if type_info['sexo'] == 'x' or type_info['sexo'] == sex:
                # Verifica profesión
                if type_info['profesion'] == 'x' or profession in professions.get(type_info['profesion'], []):
                    return type_info['tipo']
    return 'No type found'

# Cargar tipos de usuario
user_types = load_user_types('data/user_types.txt')

# Leer la información de los usuarios y determinar su tipo
results = []
with open('data/users.txt', 'r', newline='') as file:
    reader = csv.reader(file, delimiter='\t')
    for row in reader:
        user_id = row[0]
        age = int(row[1])
        sex = row[2].lower()  # Asegurarse de que todo está en minúsculas para la comparación
        profession = row[3].lower()  # Asegurarse de que todo está en minúsculas para la comparación
        user_type = determine_user_type(age, sex, profession, user_types)
        results.append([user_id, user_type])

# Escribir los resultados en un nuevo archivo
with open('data/user_types_assigned.txt', 'w', newline='') as file:
    writer = csv.writer(file, delimiter='\t')
    writer.writerows(results)

print("El procesamiento ha finalizado y los resultados están en 'user_types_assigned.txt'")


El procesamiento ha finalizado y los resultados están en 'user_types_assigned.txt'


Generador de vectores basados en contenido y colaborativos

In [1]:
import pandas as pd
import numpy as np

# Cargar datos de u1_base.txt
ratings_data = pd.read_csv("data/u1_base.txt", sep='\t', names=['UserID', 'MovieID', 'Rating'], encoding="utf-8")

# Cargar datos de items.txt y asegurar que las columnas de géneros se lean como enteros
items_data = pd.read_csv("data/items.txt", sep='\t', names=['MovieID'] + [f'Genre_{i}' for i in range(1, 20)] + ['TitleYear'], encoding="latin-1")

# Convertir las columnas de géneros a enteros
genre_columns = [f'Genre_{i}' for i in range(1, 20)]
items_data[genre_columns] = items_data[genre_columns].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)

# Filtrar las películas que han sido puntuadas con un 3 o más
high_rated_movies = ratings_data[ratings_data['Rating'] >= 3]

# Crear un diccionario para almacenar los vectores de preferencias de cada usuario
user_preferences = {}

# Iterar sobre cada usuario y calcular su vector de preferencias
for user_id in high_rated_movies['UserID'].unique():
    # Obtener las películas puntuadas con 3 o más por el usuario actual
    user_ratings = high_rated_movies[high_rated_movies['UserID'] == user_id]
    
    # Inicializar un vector de preferencias de 19 géneros con ceros
    preference_vector = np.zeros(19)
    
    # Iterar sobre las películas puntuadas por el usuario
    for _, row in user_ratings.iterrows():
        movie_id = row['MovieID']
        rating = row['Rating']
        multiplier = rating - 2  # 3 -> 1, 4 -> 2, 5 -> 3
        
        # Asegurarse de que el ID de la película exista en items_data
        if movie_id in items_data['MovieID'].values:
            movie_genres = items_data[items_data['MovieID'] == movie_id].iloc[0, 1:20].values.astype(int)
            preference_vector += movie_genres * multiplier
    
    # Normalizar el vector de preferencias a valores entre 0 y 100
    max_value = preference_vector.max()
    if max_value > 0:
        preference_vector = (preference_vector / max_value) * 100
        
    
    # Almacenar el vector de preferencias en el diccionario
    user_preferences[user_id] = preference_vector

# Guardar los vectores de preferencias en VectoresColaborativo.txt
with open('data/VectoresColaborativos.txt', 'w') as f:
    for user_id, preferences in user_preferences.items():
        preferences = preferences.astype(int)
        preferences = np.where(preferences == 0, 1, preferences)  # Cambiar 0 a 1
        preferences_str = '\t'.join(map(str, preferences))  # Convertir a enteros y luego a strings
        f.write(f"{user_id}\t{preferences_str}\n")

# Guardar los vectores de preferencias en VectoresBasadosContenido.txt
with open('data/VectoresBasadosContenido.txt', 'w') as f:
    for user_id, preferences in user_preferences.items():
        # Mantener solo los 6 valores más altos
        top_indices = preferences.argsort()[-6:][::-1]
        top_preferences = np.zeros(19)
        top_preferences[top_indices] = preferences[top_indices]
        
        preferences_str = '\t'.join(map(str, top_preferences.astype(int)))  # Convertir a enteros y luego a strings
        f.write(f"{user_id}\t{preferences_str}\n")



[  0.          38.39285714  14.28571429   9.82142857   8.03571429
  70.53571429  15.17857143  12.5        100.           0.89285714
   2.67857143   4.46428571   3.57142857   7.14285714  43.75
  36.60714286  37.5         17.85714286   1.78571429]
[  0.          20.           4.44444444   4.44444444   8.88888889
  35.55555556  28.88888889   0.         100.           0.
   6.66666667   2.22222222   2.22222222   8.88888889  60.
   2.22222222  37.77777778   4.44444444   0.        ]
[  0.    37.5   25.     0.     0.    50.    31.25  18.75 100.     0.
   0.     0.     0.    31.25  18.75  37.5   37.5   18.75   0.  ]
[  0.          83.33333333  25.           0.           0.
  50.          66.66666667  25.          75.           0.
   0.           0.          25.          58.33333333   8.33333333
  41.66666667 100.          16.66666667   0.        ]
[  0.          81.81818182  54.54545455  32.72727273  18.18181818
 100.          16.36363636   0.          20.           0.
   0.          27.272727

Generador de vecinos

In [6]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr

# Parámetros
min_neighbors = 10
max_neighbors = 30
affinity_threshold = 0.85

# Cargar los datos de VectoresColaborativos.txt
data = pd.read_csv('data/VectoresColaborativos.txt', sep='\t', header=None)
data.columns = ['UserID'] + [f'Genre_{i}' for i in range(1, 20)]

# Obtener la lista de usuarios y las preferencias
user_ids = data['UserID'].values
preferences = data.iloc[:, 1:].values

# Crear un diccionario para almacenar los vecinos de cada usuario
neighbors = {}

# Calcular el coeficiente de correlación de Pearson entre las filas de la matriz
for i, user_id in enumerate(user_ids):
    user_pref = preferences[i]
    user_neighbors = []
    
    for j, other_user_id in enumerate(user_ids):
        if user_id != other_user_id:
            other_user_pref = preferences[j]
            correlation, _ = pearsonr(user_pref, other_user_pref)
            user_neighbors.append((other_user_id, correlation))
    
    # Ordenar los vecinos por afinidad (mayor correlación)
    user_neighbors.sort(key=lambda x: x[1], reverse=True)
    
    # Filtrar los vecinos según el umbral y el rango de número de vecinos
    filtered_neighbors = []
    for neighbor_id, correlation in user_neighbors:
        if len(filtered_neighbors) < min_neighbors or (len(filtered_neighbors) < max_neighbors and correlation >= affinity_threshold):
            filtered_neighbors.append((neighbor_id, correlation))
        else:
            break
    
    neighbors[user_id] = filtered_neighbors

# Guardar los vecinos en un archivo Vecinos.txt
with open('data/Vecinos.txt', 'w') as f:
    for user_id, user_neighbors in neighbors.items():
        neighbors_str = '\t'.join([f"{neighbor_id}:{correlation:.4f}" for neighbor_id, correlation in user_neighbors])
        f.write(f"{user_id}\t{neighbors_str}\n")